## First: Install Midiapipe

In [2]:
# !pip install mediapipe

  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)


In [29]:
import cv2
import time
import os
import mediapipe as mp

# Class for detection of hands

In [26]:
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
 
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
 
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
 
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)
        return img
 
    def findPosition(self, img, handNo=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                # print(id, cx, cy)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 105, (255, 0, 255), cv2.FILLED)
 
        return lmList

## Configurations of video

In [24]:
# Configurations
fontColor = (255, 0, 0);
fontSize = 2;
fontBody = 3;
FPSPosition = (5, 30);
NumberPosition = (5, 90);

pTime = 0
tipIds = [4, 8, 12, 16, 20]

detector = handDetector(detectionCon=0.75)

## Start the application

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # captures the video frame
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
 
    if len(lmList) != 0:
        fingers = []
 
        # Thumb
        if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
 
        # 4 Fingers
        for id in range(1, 5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
 
        totalFingers = fingers.count(1)
        
        # shows the predicted number
        cv2.putText(img, 
                    'Number: '+str(totalFingers), 
                    NumberPosition, 
                    cv2.FONT_HERSHEY_PLAIN, 
                    fontSize, 
                    fontColor, 
                    fontBody
                   )
    # show the FPS
    fps = 1 / (time.time() - pTime)
    pTime = time.time()
    cv2.putText(img, 
                f'FPS: {int(fps)}', 
                FPSPosition, 
                cv2.FONT_HERSHEY_PLAIN, 
                fontSize, 
                fontColor, 
                fontBody
               )
 
    cv2.imshow("Image", img)
    
    # Press "Q" to close windows-video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()